In [1]:
# Melakukan impor libraries yang diperlukan untuk pembuatan kelompok cross validation.
import pandas as pd
import time
import datetime
import random

In [2]:
# Mencatat waktu dimulainya keseluruhan program pembuatan kelompok cross validation.
global_start_time = time.time()

In [3]:
# Membaca sekaligus menampilkan isi dari data training.
df_train = pd.read_csv('../input/bdc-satriadata2021/train.csv')
df_train

,nomor,jenis kelamin,usia
0,1,0,27
1,2,1,24
2,3,0,29
3,4,1,23
4,5,0,20
...,...,...,...
765,766,0,21
766,767,0,25
767,768,0,21
768,769,0,23


In [4]:
# Menetapkan seed untuk reproduksibilitas.
seed = 2021

# Menetapkan jumlah kelompok cross validation.
fold_number = 5

In [5]:
# Memisahkan dataframe untuk dua jenis kelamin berbeda.
perempuan = df_train[df_train.iloc[:, 1] == 0].sort_values('usia').reset_index()
laki_laki = df_train[df_train.iloc[:, 1] == 1].sort_values('usia').reset_index()

# Mencatat indeks foto bermasalah.
nomor_bermasalah = [104-1, 260-1, 290-1, 295-1, 767-1]

# Menyusun daftar indeks yang tidak bermasalah.
safe_pict_list_0 = []
for element in list(perempuan['index']):
    if element not in nomor_bermasalah:
        safe_pict_list_0.append(element)

safe_pict_list_1 = []
for element in list(laki_laki['index']):
    if element not in nomor_bermasalah:
        safe_pict_list_1.append(element)

# Menyiapkan daftar pengelompokan indeks untuk keperluan cross validation.
fold_dict = {integer: [] for integer in range(5)}

# Memanggil seed untuk reproduksibilitas.
random.seed(seed)

# Membagi seluruh indeks menjadi lima sama besar.
remove_size_0 = len(safe_pict_list_0) % fold_number
iteration_size_0 = len(safe_pict_list_0) // fold_number
shuffle_list_0 = safe_pict_list_0.copy()
random.shuffle(shuffle_list_0)
removed_0 = shuffle_list_0[:remove_size_0]
for element in removed_0:
    safe_pict_list_0.remove(element)
for iteration in range(iteration_size_0):
    local_part = safe_pict_list_0[iteration*fold_number: (iteration+1)*fold_number].copy()
    random.shuffle(local_part)
    for sub_element in range(fold_number):
        fold_dict[sub_element].append(local_part[sub_element])

remove_size_1 = len(safe_pict_list_1) % fold_number
iteration_size_1 = len(safe_pict_list_1) // fold_number
shuffle_list_1 = safe_pict_list_1.copy()
random.shuffle(shuffle_list_1)
removed_1 = shuffle_list_1[:remove_size_1]
for element in removed_1:
    safe_pict_list_1.remove(element)
for iteration in range(iteration_size_1):
    local_part = safe_pict_list_1[iteration*fold_number: (iteration+1)*fold_number].copy()
    random.shuffle(local_part)
    for sub_element in range(fold_number):
        fold_dict[sub_element].append(local_part[sub_element])

# Memasukkan kembali indeks yang tersisa.
leftovers = removed_0 + removed_1
random.shuffle(leftovers)
random.shuffle(nomor_bermasalah)

for fold in range(fold_number):
    fold_dict[fold].append(leftovers[fold])
    fold_dict[fold].append(nomor_bermasalah[fold])

# Menysusun indeks gambar dan kelompok cross validationnya.
fold_index = {}
for element_0 in fold_dict:
    for element_1 in fold_dict[element_0]:
        fold_index[element_1] = element_0
sorted_fold_index = [fold_index[integer] for integer in range(len(fold_index))]

# Menempatkan indeks gambar pada tiap kelompok cross validationnya.
df_train['fold'] = sorted_fold_index

In [6]:
# Melipatgandakan nilai pada fold, jenis kelamin, dan usia menjadi sebanyak tiga kali lipat.
triple_fold = []
for element in sorted_fold_index:
    for iteration in range(3):
        triple_fold.append(element)

jenis_kelamin = []
for element in list(df_train['jenis kelamin']):
    for iteration in range(3):
        jenis_kelamin.append(element)

usia = []
for element in list(df_train['usia']):
    for iteration in range(3):
        usia.append(element)

In [7]:
# Membaca sekaligus menampilkan informasi wajah pada data training.
info_train = pd.read_csv('../input/bdc-sd2021-data-tambahan/train_facial_data.csv')
info_train['fold'] = triple_fold
info_train['jenis_kelamin'] = jenis_kelamin
info_train['usia'] = usia
info_train = info_train[info_train.iloc[:, 2] != -1].reset_index().drop('index', axis = 1)
info_train

,nomor,urutan_gambar,x_coor,y_coor,width,height,x_lefteye,y_lefteye,x_righteye,y_righteye,x_nose,y_nose,x_mouthleft,y_mouthleft,x_mouthright,y_mouthright,fold,jenis_kelamin,usia
0,1,1,263,176,315,431,329,357,475,348,393,448,337,494,491,484,3,0,27
1,1,2,460,161,229,304,499,295,588,283,531,355,517,393,611,381,3,0,27
2,1,3,455,701,153,203,489,789,557,777,524,826,504,855,569,841,3,0,27
3,2,1,592,489,108,138,622,541,671,547,639,567,614,585,667,591,0,1,24
4,2,2,472,407,182,252,517,510,599,509,549,557,511,587,601,591,0,1,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300,769,2,323,157,297,315,395,322,490,265,456,370,467,429,544,380,4,0,23
2301,769,3,382,504,360,422,486,724,619,642,602,780,572,854,689,787,4,0,23
2302,770,1,422,163,379,563,527,385,710,415,593,518,515,591,654,612,4,0,27
2303,770,2,248,180,296,403,303,374,411,321,358,424,369,506,451,467,4,0,27


In [8]:
# Mencatat informasi penting seputar target dan kelompok cross validation.
important_info = info_train[['nomor', 'urutan_gambar', 'jenis_kelamin', 'usia','fold']]
important_info.to_csv('train_target_and_fold.csv', index = False)

In [9]:
# Mencatat waktu berakhirnya keseluruhan program pembuatan kelompok cross validation.
global_end_time = time.time()

# Menampilkan waktu eksekusi dari keseluruhan program pembuatan kelompok cross validation.
total_execution_time = datetime.timedelta(seconds = global_end_time - global_start_time)
print("total execution time: %s" % (total_execution_time))

total execution time: 0:00:00.254951
